In [57]:
from openai import OpenAI
import json
from threading import Lock
from datetime import datetime

In [56]:
lock = Lock()

In [41]:
client = OpenAI(
    # #将这里换成你在aihubmix api keys拿到的密钥
    api_key="sk-4CFE7AOiCwVbiRXw65A86b969f204a98B4Dd5bB0D97c9b70",
    # 这里将官方的接口访问地址，替换成aihubmix的入口地址
    base_url="https://aihubmix.com/v1"
)

# client = OpenAI(
#     # #将这里换成你在aihubmix api keys拿到的密钥
#     api_key="LA-dbd61865236547dea7f11d76668f54d25f9c51c40e8d41109a67dbe692110a13",
#     # 这里将官方的接口访问地址，替换成aihubmix的入口地址
#     base_url="https://api.llama-api.com"
# )

In [42]:
def message_construct(roomId): 
  JSON_FILE = f'data/room/{roomId}.json'
  with open(JSON_FILE, "r") as file:
    data = json.load(file)
    topic = data["topic"]
    member = []
    for item in data['roomMember']:
      member.append(item['memberName'])
    prompt = f'''You are an experienced instructor specialized in facilitating collaborative learning and developing students shared metacognition using the community of inquiry framework. I will provide you with a conversation history and an explanation about the history at this moment. The conversation history includes relevant decisions and explanations about previous stages. You need to identify which stage the current conversation is in by analyzing all these materials.
              The topic is {topic}.
              The group members are {' '.join(f'{item},' for item in member)}.
              Below are the five stages in the community of inquiry framework:
              Stage 1. Problem Defining Stage
              Stage Content: Students identify the topic of discussion, clarify the task objectives, and the core of the problem.
              Purpose: Ensure all participants are on the same page and stimulate interest and enthusiasm for the discussion.
              Stage 2. Exploration
              Stage Content: Students explore multiple viewpoints and solutions through discussion, research, and sharing of information.
              Purpose: To broaden thinking, stimulate creativity, and allow students to freely propose and explore different ideas.
              Stage 3. Integration
              Stage Content: Students organize, compare, and analyze the information obtained in the exploration phase to form a systematic understanding.
              Purpose: Improve students critical thinking skills and help them form a comprehensive perspective on the problem.
              Stage 4. Resolution
              Stage Content: Students apply the integrated knowledge, propose specific solutions or action plans, and start to implement them.
              Purpose: To realize the application of knowledge and encourage students to solve real problems in practice.
              Stage 5. Feedback
              Stage Content: Students summarize and reflect on the whole process, evaluate learning outcomes, and assess the effectiveness of the discussion.
              Purpose: Promote self-reflection and collective reflection, help students recognize the strengths and weaknesses of the learning process, and provide a basis for improvement in future discussions.
              Task:
                  Identify which stage the conversation is in at this moment and explain your decision by analyzing the conversation content and matching it with the given five stages.
                  Based on the identified stage and explanation, determine whether to intervene in this discussion. 
                  If need to intervene, give your instruction to guide the discussion and help them find a practical solution, meanwhile you also need to increase the low engagement student learning interest in this dialogue perfromance.
            '''
    history = data["history"]
    messages=[
      {"role": "system", "content": prompt}
    ]
    for item in history:
      if item["userId"]=="u001":
        msg_item = {"role":"assistant","content":item["text"]}
      else:
        msg_item = {"role":"user","name":item["userName"],"content":item["text"]}
      messages.append(msg_item)  
  return messages

In [43]:
roomId = 'cCwZpS'


In [44]:
new_messages = message_construct(roomId)

In [45]:
new_messages

[{'role': 'system',
  'content': "You are an experienced instructor specialized in facilitating collaborative learning and developing students shared metacognition using the community of inquiry framework. I will provide you with a conversation history and an explanation about the history at this moment. The conversation history includes relevant decisions and explanations about previous stages. You need to identify which stage the current conversation is in by analyzing all these materials.\n              The topic is How do you view college students' use of generative AI? Please choose a perspective to discuss..\n              The group members are Jim, Alice, Bob, Jack, David,.\n              Below are the five stages in the community of inquiry framework:\n              Stage 1. Problem Defining Stage\n              Stage Content: Students identify the topic of discussion, clarify the task objectives, and the core of the problem.\n              Purpose: Ensure all participants are 

In [46]:
tools = [
    {
        "type": "function",
        "function":
            {
                "name": "generate_response_fields",
                "description": "Automatically decide whether to intervene and generate response details.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "intervene_required": {
                            "type": "boolean",
                            "description": "Indicates whether an intervene is required.",
                        },
                        "identified_stage": {
                            "type": "string",
                            "description": "The stage of the discussion",
                        },
                        "stage_explanation": {
                            "type": "string",
                            "description": "The reason Why this stage",
                        },
                        "response_content": {
                            "type": "string",
                            "description": "The content of the response if a intervene is required.",
                        },
                        "response_target": {
                            "type": "string",
                            "description": "The target of the response, i.e., member names",
                        },
                    },
                    "required": ["respond_required"],
                },
            }
    }
]

In [47]:
# response = client.chat.completions.create(
#   model="llama3.1-70b",
#   messages=new_messages,
#   tools=tools
# )
response = client.chat.completions.create(
  model="gpt-4o",
  messages=new_messages,
  tools=tools
)

In [48]:
response

ChatCompletion(id='chatcmpl-AqbpLl8QJXDjKrBi34s52AFesZN9Z', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yMvARoPBTBcDF8XcqsqP7G4N', function=Function(arguments='{"identified_stage":"Stage 1. Problem Defining Stage","intervene_required":true,"response_content":"It seems like we\'re in the initial stage of the discussion, focusing on clarifying the perspective from which we\'ll address the topic. Jim and Alice have started exploring perspectives but haven\'t settled on one that aligns well with the topic. Let\'s try to combine your ideas to find a more focused perspective. \\n\\nFor example, \\"How reliable is the use of generative AI by college students in their academic work?\\" or \\"How does generative AI impact college students\' learning experience?\\" These perspectives might allow us to better meet th

In [49]:
results = response.choices[0].message.tool_calls

In [53]:
json.loads(results[0].function.arguments)

{'identified_stage': 'Stage 1. Problem Defining Stage',
 'intervene_required': True,
 'response_content': 'It seems like we\'re in the initial stage of the discussion, focusing on clarifying the perspective from which we\'ll address the topic. Jim and Alice have started exploring perspectives but haven\'t settled on one that aligns well with the topic. Let\'s try to combine your ideas to find a more focused perspective. \n\nFor example, "How reliable is the use of generative AI by college students in their academic work?" or "How does generative AI impact college students\' learning experience?" These perspectives might allow us to better meet the topic\'s objective. \n\nRemember, the aim is to ensure our discussions align closely with the topic and each contribution helps us advance towards an understanding or a solution. \n\nCould Jim or Alice suggest a revised perspective that combines both elements?',
 'response_target': 'Jim, Alice, Bob',
 'stage_explanation': 'The conversation re

In [60]:
def save_response(roomId,results):
  with lock:
    JSON_FILE = f'data/room/{roomId}.json'
    with open(JSON_FILE, "r") as file:
      old_data = json.load(file)
      for i in results:
        item = json.loads(i.function.arguments)
        msg = {
          "userId": "u000",
          "userName": "Agent",
          "userAvatar": "/src/assets/Agent.PNG",
          "text": item["response_content"],
          "time": datetime.now().strftime("%I:%M:%S %p"),
          "roomId": roomId
        }
        old_data["history"].append(msg)
      # 4. 将更新后的数据写回文件
      with open(JSON_FILE, 'w') as f:
        json.dump(old_data, f, ensure_ascii=False, indent=2)
    

In [61]:
save_response(roomId,results)

In [38]:
# print(response.model_dump_json(indent=2))
result = json.loads(response.choices[0].message.tool_calls[0].function.arguments)

In [39]:
result

{'respond_required': True,
 'response_content': "Since several of you have mentioned integrating the use of generative AI with the context of college students, let's consider discussing how generative AI can support college students in their academic and personal development. This can include aspects like enhancing learning experiences, assisting in daily tasks, or even managing stress. This perspective ties in both the technology and its user base effectively.",
 'response_target': 'all'}

In [24]:
result

{'respond_required': True,
 'response_content': "Jim and Bob, you both seem to have great ideas. Jim, you’re pointing out an important area about reliability, and Bob, you're thinking about relevance. Let's try merging these insights into a comprehensive perspective: the impact of AI reliability on student learning routines and outcomes.",
 'response_target': 'Jim and Bob'}